In [2]:
#Checking for availability of GPU + specifications
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 12 00:04:53 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.12                 Driver Version: 552.12         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:02:00.0 Off |                  N/A |
| N/A   40C    P8              2W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#
## Fine - Tuning Whisper with HuggingFace Transformer

Transformers have come up as a promising alternative, offering pre-trained parameters and fine-tuning capabilities. The Whisper model by OpenAI is trained on extensive multilingual data, has become a benchmark for ASR. This incorporation leverages the transformer's self-attention mechanism to process temporal sequences in audio data, providing extensive improvements over conventional models in terms of parallelizable training and handling long-term dependencies.
This model is also highly effective for noise robustness and language support.

In [4]:
#Suite for handling multimedia files and streams
!pip install ffmpeg-python --user

In [5]:
!pip install git+https://github.com/huggingface/datasets
!pip install git+https://github.com/huggingface/transformers
!pip install librosa --user
!pip install evaluate>=0.3.0 --user
!pip install jiwer --user
!pip install gradio --user
!pip install more-itertools --user

  Cloning https://github.com/huggingface/datasets to c:\users\shiva\appdata\local\temp\pip-req-build-pjspgi_a
  Resolved https://github.com/huggingface/datasets to commit af3acfdfcf76bb980dbac871540e30c2cade0cf9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pyarrow-16.1.0-cp311-cp311-win_amd64.whl.metadata (3.1 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
Using cached pyarrow-16.1.0-cp311-cp311-win_amd64.whl (25.9 MB)
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
  Created wheel for datasets: filename=datasets-2.19.3.dev0-py3-none-any.whl size=524354 sha256=46747fa

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets 'C:\Users\shiva\AppData\Local\Temp\pip-req-build-pjspgi_a'
ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\tqdm.exe' -> 'C:\\Python311\\Scripts\\tqdm.exe.deleteme'



  Cloning https://github.com/huggingface/transformers to c:\users\shiva\appdata\local\temp\pip-req-build-jur1gjdr
  Resolved https://github.com/huggingface/transformers to commit 35a6d9d6483d4d4d7cd817ed4ecfd5f86e1f9a23
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9156712 sha256=72a095ab639c97190ffad960f978b26f79a6b37faeb2c0b4b3aa2acadd240cbd
  Stored in directory: C:\Users\shiva\AppData\Local\Temp\pip-ephem-wheel-cache-1fias604\wheels\04\a3\f1\b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built 

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\shiva\AppData\Local\Temp\pip-req-build-jur1gjdr'
ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\transformers-cli.exe' -> 'C:\\Python311\\Scripts\\transformers-cli.exe.deleteme'



In [6]:
#Linking to HuggingFace Hub using access token
from huggingface_hub import notebook_login

notebook_login()

#
-- Dataset: Mozilla CommonVoice - English

- Whisper has 5 configurations with different model sizes
- Using Whisper 'small' model as per available computing power

In [7]:
from datasets import interleave_datasets, load_dataset

def load_streaming_dataset(dataset_name, dataset_config_name, split, **kwargs):
    '''
    Streaming mode for loading data progressively while iterating over dataset.

    Function downloads fragments of data and combines (concatenation) multiple of them.
    '''
    if "+" in split:
        # load multiple splits with streaming mode
        dataset_splits = [load_dataset(dataset_name, dataset_config_name, split=split_name, streaming=True, **kwargs) for split_name in split.split("+")]
        # interleave to form one dataset
        interleaved_dataset = interleave_datasets(dataset_splits)
        return interleaved_dataset
    else:
        # single split with streaming mode
        dataset = load_dataset(dataset_name, dataset_config_name, split=split, streaming=True, **kwargs)
        return dataset

In [8]:
from datasets import IterableDatasetDict

raw_datasets = IterableDatasetDict()


raw_datasets["train"] = load_streaming_dataset("mozilla-foundation/common_voice_11_0", "en", split="train", use_auth_token=True, trust_remote_code=True)
raw_datasets["test"] = load_streaming_dataset("mozilla-foundation/common_voice_11_0", "en", split="test", use_auth_token=True, trust_remote_code=True)


C:\Users\shiva\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:2552: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
C:\Users\shiva\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:2552: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


##
AUDIO PRE-PROCESSING

In [9]:
from transformers import WhisperProcessor

# Whisper Processor works as feature extractor and tokenizer which performs audio pre-processing and further processes text token
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

raw_datasets["train"].features

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'client_id': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None),
 'up_votes': Value(dtype='int64', id=None),
 'down_votes': Value(dtype='int64', id=None),
 'age': Value(dtype='string', id=None),
 'gender': Value(dtype='string', id=None),
 'accent': Value(dtype='string', id=None),
 'locale': Value(dtype='string', id=None),
 'segment': Value(dtype='string', id=None)}

In [10]:
from datasets import Audio


raw_datasets = raw_datasets.cast_column("audio", Audio(sampling_rate=16000)) #sampling rate for Whisper

from transformers.models.whisper.english_normalizer import BasicTextNormalizer

do_lower_case = False
do_remove_punctuation = False

normalizer = BasicTextNormalizer()

def prepare_dataset(batch):
    audio = batch["audio"]

    #log-Mel Spectrogram: input features
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    #pre-processing
    transcription = batch["sentence"]
    if do_lower_case:
        transcription = transcription.lower()
    if do_remove_punctuation:
        transcription = re.sub(punctuation_to_remove_regex, " ", transcription).strip()

    batch["labels"] = processor.tokenizer(transcription).input_ids # target text -> label id
    return batch

In [11]:
#Prepearing Dataset for Training
vectorized_datasets = raw_datasets.map(prepare_dataset, remove_columns=list(next(iter(raw_datasets.values())).features)).with_format("torch")

vectorized_datasets["train"] = vectorized_datasets["train"].shuffle(
    buffer_size=300,
    seed=0)

max_input_length = 15.0

def is_audio_in_length_range(length):
    return length < max_input_length

vectorized_datasets["train"] = vectorized_datasets["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"])



##
TRAINING PIPELINE

In [14]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    ''' Data Collator prepares PyTorch tensors for the model'''

    processor: Any
    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels as per length -> Torch Tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # padding to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [15]:
# WER (Word Error Rate) Metric

import evaluate
metric = evaluate.load("wer")

do_normalize_eval = True

In [16]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]

        pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
        label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer":wer}

#
LOADING WHISPER 'SMALL' CHECKPOINT

In [17]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

In [18]:
!pip install transformers[torch]
!pip install -U accelerate
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.41.2', '0.31.0')

In [19]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

2.3.1+cu118
True
CUDA version: 11.8
0
NVIDIA GeForce GTX 1650


#
DETAILS OF TRAINING PARAMETERS

In [20]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./Whisper-Small",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=200,
    eval_steps=200,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [22]:
from transformers import TrainerCallback
from transformers.trainer_pt_utils import IterableDatasetShard
from torch.utils.data import IterableDataset
'''
Callbacks can customize and inspect the behaviour of training loop in Trainer - PyTorch
'''
class ShuffleCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, train_dataloader, **kwargs):
        if isinstance(train_dataloader.dataset, IterableDatasetShard):
            pass
        elif isinstance(train_dataloader.dataset, IterableDataset):
            train_dataloader.dataset.set_epoch(train_dataloader.dataset._epoch + 1)


In [23]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    callbacks=[ShuffleCallback()],
)


max_steps is given, it will override any value given in num_train_epochs


In [24]:
#For non-default generation parameters set in model config (warning)
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_length=448,
    suppress_tokens=[],
    begin_suppress_tokens=[220, 50257]
)

model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)
generation_config.save_pretrained(training_args.output_dir)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


In [ ]:
trainer.train()

Reading metadata...: 948736it [01:10, 13540.75it/s]
C:\Users\shiva\AppData\Roaming\Python\Python311\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Python311\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:691: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub()

## Building a Demo

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="Shangyy/Whisper-Small")

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

#Gradio - Microphone transcription (Another block can be added for separate file uploads)
demo = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Voice-to-Text System",
    description="Realtime demo for transcribing spoken language into text.",
)

demo.launch()